# R-CNN 

성능 : pascal VOC 2010을 기준으로 53.7%, 이미지 한 장의 CPU로는 47, GPU 13초

## R-CNN이 Object-Detection을 수행하는 알고리즘 

1. 입력 이미지에 Selective Search 알고리즘을 적용하여 물체가 있을 만한 박스 2000 개 를 추출
2. 모든 박스를 227 x 227 크기로 리사이즈 warp 한다. (박스의 비율은 고려하지 않음)
3. 미리 이미지넷 데이터를 통해 학습 시켜놓은 CNN을 통과시켜 4096 차원의 특징 백터를 추출 
4. 추출된 백터를 가지고 각각의 클래스 (object의 종류) 마다 학습시켜놓은 SVM Classifier를 통과한다.
5. 바운딩 박스 리그레이션을 적용하여박스의 위치를 조정한다


### 1. Region Proposal

주어진 이미지에서 물체가 있을법한 위치를 찾는 것 // class와 무관

여기서는 Selective search라는 룰 베이스 알고리즘을 적용 

주변 픽셀 간의 유사도 기준으로 Segmentation을 만들고 
2000 개의 물체가 있을 법한 박스 찾음

but 현재는 이것도 NN에서 수행 

### 2. Feature Extraction

Selective Search를 통해서 찾아낸 2000 개의 박스영역은 227 x 227 크기로 리사이즈 (warp)

그후 Image classfication으로 미리 학습되어있는 CNN 모델을통과하여 4096 크기의 특징 백터를 추출한다.

이미지넷 데이터 (ILSVR 2012 classification)으로 미리학습된 CNN 모델을 가져온후 fine tune 하는 방식
저자들은 Object Detection을 적용할 데이터셋에서 ground truth에 해당하는 이미지들을 가져와 학습 시킴 
그 후 classification의 마지막 레이어를 objectDetection 클래스 수 N과 아무 물체도 포함되지 않는 배경 까지 n+1 로 맞춤 

파인 튜닝(Fine-tuning)
모델의 파라미터를 미세하게 조정하는 행위
특히 딥러닝에서는 이미 존재하는 모델에 추가 데이터를 투입하여 파라미터를 업데이트하는 것을 말한다.


### 3. Classification 

CNN을 통해 추출한 백터를 각각의 클래스 별로 SVM Classifier를 학습 시킴
주어진 백터를 놓고 이것이 해당 물체가 맞는지 구분하는 Classifier 모델을 학습 시킴

각각의 박스들은 어떤 물체일 확률 Score 값을 갖게 됨

### 4. Non-Maximum Suppression

가장 높은 score 박스만 남기고 나머지는 제거
IOU값이 가장 높은 물체를 선택하는 절차 

논문에서는 IoU가 0.5보다 크면 동일한 물체를 대상으로 한박스로 판단 및 적용


### 5. Bounding box Regression

selective search를 통해서 찾은 박스 위치는 부정확하다.
성능을 끌어 올리기 위해서 박스 위치를 교정해 주어야하는데 여기서 한다.

박스는 x, y, w, h 로 표기할 수있는데 각각

> 측정 값
p^i = (p^i_x, p^i_y, p^i_w, p^i_h)

> Ground Truth
G = (G_x, G_y, G_w, G_h)

목표는 P 값을 G에 가깝도록 이동 시키는 함수를 학습 

이절차에서는 바운딩 박스를 조정해 주는 선형 회기를 학습

R-CNN에서 학습이 일어나는 부분

1. 이미지 넷으로 이미 학습된 모델을 가져와 fine tuning 하는 부분
2. SVM Classifier를 학습시키는 부분
3. Bounding Box Regression

In [ ]:
문제점 

R-CNN의 가장 큰 문제는 복잡한 프로세스로 인한 과도한 연산이 있다.
CPU 연산에 의존하는 작업들이 많음 

참고 

[1] https://yeomko.tistory.com/13

[2] https://gist.github.com/aisolab/46f73657c7fcbec91004708e01a67ba0